In [ ]:
%pip install matplotlib

In [ ]:
import numpy as np
from decayangle.lorentz import LorentzTrafo
from decayangle.decay_topology import Topology
from decayangle.config import config as cfg
import matplotlib.pyplot as plt

def make_four_vectors(phi_rf, theta_rf, psi_rf):
    import numpy as np

    # Make sure, the sorting is turned off
    cfg.sorting = "off"

    # Given values
    # Lc -> p K pi
    m1, m2, m3, m0 = 0.93827, 0.493677, 0.139570, 2.28646
    m12 = 2.756020646168232**0.5
    m23 = 1.3743747462964881**0.5

    # Squared masses
    m0sq, m1sq, m2sq, m3sq, m12sq, m23sq = [x**2 for x in [m0, m1, m2, m3, m12, m23]]

    # Källén function
    def Kallen(x, y, z):
        return x**2 + y**2 + z**2 - 2 * (x * y + x * z + y * z)

    # Calculating missing mass squared using momentum conservation
    m31sq = m0sq + m1sq + m2sq + m3sq - m12sq - m23sq

    # Momenta magnitudes
    p1a = np.sqrt(Kallen(m23sq, m1sq, m0sq)) / (2 * m0)
    p2a = np.sqrt(Kallen(m31sq, m2sq, m0sq)) / (2 * m0)

    # Directions and components
    cos_zeta_12_for0_numerator = (m0sq + m1sq - m23sq) * (
        m0sq + m2sq - m31sq
    ) - 2 * m0sq * (m12sq - m1sq - m2sq)
    cos_zeta_12_for0_denominator = np.sqrt(Kallen(m0sq, m2sq, m31sq)) * np.sqrt(
        Kallen(m0sq, m23sq, m1sq)
    )
    cos_zeta_12_for0 = cos_zeta_12_for0_numerator / cos_zeta_12_for0_denominator

    p1z = -p1a
    p2z = -p2a * cos_zeta_12_for0
    p2x = np.sqrt(p2a**2 - p2z**2)
    p3z = -p2z - p1z
    p3x = -p2x

    # Energy calculations based on the relativistic energy-momentum relation
    E1 = np.sqrt(p1z**2 + m1sq)
    E2 = np.sqrt(p2z**2 + p2x**2 + m2sq)
    E3 = np.sqrt(p3z**2 + p3x**2 + m3sq)

    # Vectors
    p1 = np.array([0, 0, p1z, E1])
    p2 = np.array([p2x, 0, p2z, E2])
    p3 = np.array([p3x, 0, p3z, E3])

    # Lorentz transformation
    momenta = {i: p for i, p in zip([1, 2, 3], [p1, p2, p3])}
    tree1 = Topology(root=0, decay_topology=((2, 3), 1))
    
    rotation = LorentzTrafo(0, 0, 0, phi_rf, theta_rf, psi_rf)
    
    momenta_23_rotated = tree1.root.transform(rotation, momenta)
    return momenta_23_rotated

In [ ]:
N = 1000
x = np.linspace(1e-5, np.pi - 1e-5, N)
y = np.linspace(-np.pi + 1e-5, np.pi - 1e-5, N)

X, Y = np.meshgrid(x, y)
momenta_23_rotated = make_four_vectors(1.4, X, Y)

tree1 = Topology(root=0, decay_topology=((2, 3), 1))
tree2 = Topology(root=0, decay_topology=((3, 1), 2))
tree3 = Topology(root=0, decay_topology=((1, 2), 3))

In [ ]:
def plot_for_convention(convention="helicity"):
    result_full = tree1.relative_wigner_angles(tree3, momenta_23_rotated, convention=convention)
    result_psi = result_full[1].psi_rf
    result_phi = result_full[1].phi_rf
    result_theta = result_full[1].theta_rf
    result = result_psi + result_phi

    def fmod_4(val):
        intermediate = np.fmod(val/np.pi + 4,4) - 2
        intermediate[intermediate < 0] += 4
        return intermediate

    img = plt.imshow(np.sin(result/2), cmap='hot', origin='lower',
                    extent=[x.min(), x.max(), y.min(), y.max()], aspect='auto')
    plt.colorbar(label=r"$\sin \left(\frac{(\psi_{3}^{(1)}(1) + \phi_{3}^{(1)}(1))}{2}\right)$")
    plt.xlabel(r'$\theta_{rf}$', fontsize=20)
    plt.ylabel(r'$\psi_{rf}$', fontsize=20)
    plt.title(r"$\psi_{3}^{(1)}(1) + \phi_{3}^{(1)}(1)$")
    plt.savefig(f'test_{convention}.png', dpi = 800)
    plt.show()
    img = plt.imshow(np.sin(np.fmod(result_phi + 4 * np.pi, 4*np.pi)/2), cmap='hot', origin='lower',
                    extent=[x.min(), x.max(), y.min(), y.max()], aspect='auto')

    plt.colorbar(label=r"$\sin\left(\frac{\phi_{3}^{(1)}(1)}{2} \right)$")
    plt.xlabel(r'$\theta_{rf}$', fontsize=20)
    plt.ylabel(r'$\psi_{rf}$', fontsize=20)
    plt.title(r"$\phi_{3}^{(1)}(1)$")
    plt.savefig(f'test_phi_{convention}.png', dpi = 800)
    plt.show()
    img = plt.imshow(np.sin(result_psi/2), cmap='hot', origin='lower',
                    extent=[x.min(), x.max(), y.min(), y.max()], aspect='auto')

    plt.colorbar(label=r"$\sin \left( \frac{ \psi_{3}^{(1)}(1)}{2} \right)$")
    plt.xlabel(r'$\theta_{rf}$', fontsize=20)
    plt.ylabel(r'$\psi_{rf}$', fontsize=20)
    plt.title(r"$\psi_{3}^{(1)}(1)$")
    plt.savefig(f'test_psi_{convention}.png', dpi = 800)
    plt.show()

    img = plt.imshow(np.sin(result_theta/2), cmap='hot', origin='lower',
                    extent=[x.min(), x.max(), y.min(), y.max()], aspect='auto')

    plt.colorbar(label=r"$\sin \left( \frac{\theta{3}^{(1)}(1)}{2} \right)$")
    plt.xlabel(r'$\theta_{rf}$', fontsize=20)
    plt.ylabel(r'$\psi_{rf}$', fontsize=20)
    plt.title(r"$\theta{3}^{(1)}(1)$")
    plt.savefig(f'test_theta_{convention}.png', dpi = 800)
    plt.show()

The `helicity` convention is the default. Here we boost into a new system by aligning the particle momentum with the z-axis and then boosting along the z-axis. Thus the quantization axis for the spins is given by the particles momentum direction. This is why this convention is called the `helicity` convention.

In [ ]:
plot_for_convention("helicity")

Let us look at the canonical convention. Here we stay in the canonical frame as defined by Chung. Thus after a boost into the next node in the tree, we will add a rotation about the z-axis by $-\theta$. Here $\theta$ is the helicity angle in the system that is boosted out of. So the full transformation from isobar 1 frame into isobar 2 frame becomes

$R_z(\phi) R_y(\theta) B_z(\xi) R_z(-\phi) R_y(-\theta)$

This is so, that all movements between frames are **pure** boosts. I.e. we always go back to the quantization axis defined by the first frame


In [ ]:
plot_for_convention("canonical")

The `minus_phi` convention is similar to the `canonical` convention, but instead of using $\theta$ and rotation about the y-axis, $\phi$ is used and a rotation about the z-axis. This does not change the value of the spin projection though.

In [ ]:
plot_for_convention("minus_phi")